# 03. Prompt Engineering for Tool Calling

**Learning Objectives:**
- Understand how prompts affect tool calling success
- Write effective tool descriptions and parameters
- Design system prompts that guide tool usage
- Handle multi-tool scenarios effectively
- Debug and iterate on prompt designs

**Time Allocation:** 35 minutes

---

In [1]:
# Setup and imports
import sys
import json
sys.path.append('../Day2')

from api_utils import call_openrouter, extract_function_call, extract_text_response

# Import display utilities for markdown rendering
from IPython.display import Markdown, display
try:
    from display_utils import display_llm_response, display_markdown, display_error
    print("✅ Display utilities loaded successfully!")
except ImportError:
    print("⚠️ Display utilities not found, using basic markdown rendering")
    def display_llm_response(response, model="", title="AI Response"):
        display(Markdown(response))
    def display_markdown(text, title=None):
        if title:
            display(Markdown(f"### {title}\n\n{text}"))
        else:
            display(Markdown(text))
    def display_error(error_message, error_type="Error"):
        display(Markdown(f"**{error_type}:** {error_message}"))

# Import the multi-tool approach from Notebook 2
print("📋 Building on Notebook 2's multi-tool foundation...")
print("In this notebook, we'll enhance the simple_multi_tool_approach() with better prompts!")

print("🚀 Setup complete! Ready for prompt engineering...")

✅ Display utilities loaded successfully!
📋 Building on Notebook 2's multi-tool foundation...
In this notebook, we'll enhance the simple_multi_tool_approach() with better prompts!
🚀 Setup complete! Ready for prompt engineering...


## 1. The Prompt Problem (5 mins)

Let's see how different prompts affect **what tools are used** and **how they're used**.

### The Scenario
We have multiple tools available. The key question isn't just "Will tools be used?" but "Which tools and how effectively?"

**Learning Focus:**
- How prompts influence tool selection
- How prompts affect parameter quality
- How prompts guide tool usage patterns

In [2]:
# Define tools that better demonstrate prompt differences

# Weather tool (requires location - good for parameter examples)
basic_weather_tool = {
    "name": "get_weather",
    "description": "Get weather for location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name"
            }
        },
        "required": ["location"]
    }
}

# Knowledge search tool (optional usage - shows prompt impact)
knowledge_tool = {
    "name": "search_knowledge",
    "description": "Search for current information and facts",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query for specific information"
            }
        },
        "required": ["query"]
    }
}

# Calculator tool (optional usage - shows when math is chosen)
calculator_tool = {
    "name": "calculate",
    "description": "Perform mathematical calculations",
    "parameters": {
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "Mathematical expression to calculate"
            }
        },
        "required": ["expression"]
    }
}

def test_tool_calling(query, system_prompt, tools, title):
    """Helper function to test tool calling with different prompts"""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]
    
    response = call_openrouter(
        prompt=messages,
        model="openai/gpt-4o-2024-11-20",
        functions=tools,
        temperature=0.3,
        max_tokens=200
    )
    
    print(f"\n🔍 {title}")
    print(f"Query: '{query}'")
    
    if response.get('success'):
        # Check if tool was called
        tool_call = extract_function_call(response)
        if tool_call.get('success'):
            print(f"✅ Tool called: {tool_call['function_name']}({tool_call['arguments']})")
        else:
            text_response = extract_text_response(response)
            print(f"❌ No tool called. Response: {text_response[:100]}...")
    else:
        print(f"❌ API Error: {response.get('error')}")

print("🛠️ Test function ready with optional-usage tools!")

🛠️ Test function ready with optional-usage tools!


In [3]:
# Demo: How prompts affect tool selection and quality

# Test 1: Multi-tool scenario - see which tool gets chosen
test_tool_calling(
    query="I want to plan a weekend trip to Paris",
    system_prompt="You are a helpful assistant.",
    tools=[knowledge_tool, calculator_tool],
    title="Test 1: Generic Prompt - Trip Planning"
)

# Test 2: Same query with specific guidance that encourages tool use
test_tool_calling(
    query="I want to plan a weekend trip to Paris",
    system_prompt="You are a travel planning assistant. When users mention travel, destinations, or trip planning, always search for current information about destinations, events, weather, and travel costs. Use your search tool to find up-to-date information rather than relying on general knowledge.",
    tools=[knowledge_tool, calculator_tool],
    title="Test 2: Travel-Focused Prompt with Tool Encouragement"
)

# Test 3: Even stronger tool guidance
test_tool_calling(
    query="I want to plan a weekend trip to Paris",
    system_prompt="You are a travel expert. For ANY travel query, you MUST use the search tool to find current information about the destination. Never provide travel advice without first searching for current conditions, events, and practical information.",
    tools=[knowledge_tool, calculator_tool],
    title="Test 3: Mandatory Tool Usage Prompt"
)

# Test 4: Compare tech query responses
test_tool_calling(
    query="What's happening in tech right now?",
    system_prompt="You are a helpful assistant.",
    tools=[knowledge_tool],
    title="Test 4: Generic - Vague Tech Query"
)

# Test 5: Same query with better parameter guidance
test_tool_calling(
    query="What's happening in tech right now?",
    system_prompt="You are a tech analyst assistant. When searching for information, use specific, targeted queries that will return the most relevant and current results.",
    tools=[knowledge_tool],
    title="Test 5: Analyst Prompt - Better Parameters"
)


🔍 Test 1: Generic Prompt - Trip Planning
Query: 'I want to plan a weekend trip to Paris'
❌ No tool called. Response: Planning a weekend trip to Paris sounds exciting! Here's a guide to help you organize your trip:

--...

🔍 Test 2: Travel-Focused Prompt with Tool Encouragement
Query: 'I want to plan a weekend trip to Paris'
✅ Tool called: search_knowledge({'query': 'current weather in Paris this weekend'})

🔍 Test 3: Mandatory Tool Usage Prompt
Query: 'I want to plan a weekend trip to Paris'
✅ Tool called: search_knowledge({'query': 'current travel information for a weekend trip to Paris'})

🔍 Test 4: Generic - Vague Tech Query
Query: 'What's happening in tech right now?'
✅ Tool called: search_knowledge({'query': 'latest news in technology'})

🔍 Test 5: Analyst Prompt - Better Parameters
Query: 'What's happening in tech right now?'
✅ Tool called: search_knowledge({'query': 'latest tech news October 2023'})


In [4]:
# Exercise: Test ambiguous queries that show prompt impact

# Try these queries that could work with or without tools:
test_queries = [
    "Is Python still popular for data science?",  # Could search or use training data
    "What's 15% of 83?",  # Simple enough to do mentally or calculate
    "How do I make a good impression in job interviews?",  # General advice vs current trends
    "What's the best way to learn machine learning in 2024?"  # Timeless advice vs current info
]

print("🧪 Try these ambiguous queries with different system prompts:")
for i, query in enumerate(test_queries, 1):
    print(f"{i}. {query}")

print("\n💡 Notice: These queries can be answered from training data OR by using tools.")
print("The system prompt determines which approach the model takes!")

# Your code here - test any of the above queries with different prompts:
# Example:
# test_tool_calling("Is Python still popular for data science?", "You are a helpful assistant.", [knowledge_tool], "Generic Test")
# test_tool_calling("Is Python still popular for data science?", "You are a helpful assistant. Always search for current information about technology trends.", [knowledge_tool], "Search-Focused Test")

🧪 Try these ambiguous queries with different system prompts:
1. Is Python still popular for data science?
2. What's 15% of 83?
3. How do I make a good impression in job interviews?
4. What's the best way to learn machine learning in 2024?

💡 Notice: These queries can be answered from training data OR by using tools.
The system prompt determines which approach the model takes!


In [5]:
# Exercise: Test "Should I bring an umbrella?" query
# TODO: Run the test with both system prompts and observe the difference

# Your code here:


---

## 2. Writing Better Tool Descriptions (8 mins)

The tool description is the first thing the model sees. Let's compare bad vs good descriptions.

In [6]:
# Example: Different parameter designs for a unit converter

# Bad: Unclear parameters
bad_converter = {
    "name": "convert_units",
    "description": "Convert between different units of measurement",
    "parameters": {
        "type": "object",
        "properties": {
            "value": {
                "type": "number",
                "description": "Number to convert"
            },
            "from_unit": {
                "type": "string",
                "description": "Source unit"
            },
            "to_unit": {
                "type": "string",
                "description": "Target unit"
            }
        },
        "required": ["value", "from_unit", "to_unit"]
    }
}

# Good: Clear, specific parameters with examples
good_converter = {
    "name": "convert_units",
    "description": "Convert between different units of measurement for distance, weight, temperature, and volume",
    "parameters": {
        "type": "object",
        "properties": {
            "value": {
                "type": "number",
                "description": "The numerical value to convert (e.g., 5.5, 100, 32)"
            },
            "from_unit": {
                "type": "string",
                "description": "Source unit - use standard abbreviations: 'km', 'miles', 'kg', 'lbs', 'f', 'c', 'liters', 'gallons'"
            },
            "to_unit": {
                "type": "string",
                "description": "Target unit - use same format as from_unit (e.g., 'km', 'miles', 'kg', 'lbs', 'f', 'c')"
            }
        },
        "required": ["value", "from_unit", "to_unit"]
    }
}

display_markdown("""
### Parameter Design Principles:

1. **Be Specific**: "Source unit" vs "use standard abbreviations like 'km', 'miles'"
2. **Provide Examples**: Show the exact format you expect
3. **Stay Consistent**: Use the same abbreviation format throughout
4. **Consider Edge Cases**: What happens with unusual inputs?
""")


### Parameter Design Principles:

1. **Be Specific**: "Source unit" vs "use standard abbreviations like 'km', 'miles'"
2. **Provide Examples**: Show the exact format you expect
3. **Stay Consistent**: Use the same abbreviation format throughout
4. **Consider Edge Cases**: What happens with unusual inputs?


In [7]:
# Test parameter design differences
queries = [
    "Convert 5 kilometers to miles",
    "How many pounds is 2.5 kilograms?"
]

system_prompt = "You are a helpful assistant with unit conversion tools. Use them when users ask for conversions."

for query in queries:
    print(f"\n📏 Testing: '{query}'")
    
    # Test with bad parameters
    response = call_openrouter(
        prompt=[{"role": "system", "content": system_prompt}, {"role": "user", "content": query}],
        model="openai/gpt-4o-2024-11-20",
        functions=[bad_converter],
        temperature=0.3,
        max_tokens=150
    )
    
    if response.get('success'):
        tool_call = extract_function_call(response)
        if tool_call.get('success'):
            print(f"❌ Bad params: {tool_call['arguments']}")
        else:
            print("❌ Bad params: No tool called")
    
    # Test with good parameters
    response = call_openrouter(
        prompt=[{"role": "system", "content": system_prompt}, {"role": "user", "content": query}],
        model="openai/gpt-4o-2024-11-20",
        functions=[good_converter],
        temperature=0.3,
        max_tokens=150
    )
    
    if response.get('success'):
        tool_call = extract_function_call(response)
        if tool_call.get('success'):
            print(f"✅ Good params: {tool_call['arguments']}")
        else:
            print("✅ Good params: No tool called")


📏 Testing: 'Convert 5 kilometers to miles'
❌ Bad params: {'value': 5, 'from_unit': 'kilometers', 'to_unit': 'miles'}
✅ Good params: {'value': 5, 'from_unit': 'km', 'to_unit': 'miles'}

📏 Testing: 'How many pounds is 2.5 kilograms?'
❌ Bad params: {'value': 2.5, 'from_unit': 'kilograms', 'to_unit': 'pounds'}
✅ Good params: {'value': 2.5, 'from_unit': 'kg', 'to_unit': 'lbs'}


### Exercise: Improve These Tool Descriptions
Rewrite these tool descriptions to be more effective:

In [8]:
# Different system prompt strategies

system_prompts = {
    "generic": "You are a helpful assistant.",
    
    "tool_aware": "You are a helpful assistant with access to weather and calculation tools. Use them when appropriate.",
    
    "specific_guidance": """
You are a helpful assistant with access to weather and calculation tools.

Use the weather tool when users:
- Ask about current conditions
- Want to plan outdoor activities
- Need weather-related advice

Use the calculator tool when users:
- Need mathematical calculations
- Ask for percentages, tips, or conversions
- Want to solve numerical problems

Always use tools when the information is needed to give an accurate answer.
""",
    
    "persona_based": """
You are an outdoor activity planning assistant. You help people make informed decisions about outdoor activities.

You have access to weather tools and calculators. Use weather tools to check conditions and calculators for any planning math (distances, costs, etc.).

Always check the weather when someone asks about outdoor activities, even if they don't explicitly mention weather.
"""
}

# Define good versions of tools for this section
good_weather_tool = {
    "name": "get_weather",
    "description": "Get current weather conditions including temperature, humidity, and precipitation for planning outdoor activities",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City name with country for accurate weather lookup (e.g., 'London, UK' or 'New York, USA')"
            }
        },
        "required": ["location"]
    }
}

good_calculator = {
    "name": "calculate",
    "description": "Perform mathematical calculations for planning, budgeting, and problem-solving",
    "parameters": {
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "Mathematical expression to calculate (e.g., '47 * 0.18', '(100 + 50) / 2', '25 * 3')"
            }
        },
        "required": ["expression"]
    }
}

# Tools for testing
tools = [good_weather_tool, good_calculator]

def compare_system_prompts(query, prompts_to_test):
    """Compare how different system prompts handle the same query"""
    print(f"\n🎯 Query: '{query}'")
    print("=" * 50)
    
    for name, prompt in prompts_to_test.items():
        response = call_openrouter(
            prompt=[{"role": "system", "content": prompt}, {"role": "user", "content": query}],
            model="openai/gpt-4o-2024-11-20",
            functions=tools,
            temperature=0.3,
            max_tokens=200
        )
        
        if response.get('success'):
            tool_call = extract_function_call(response)
            if tool_call.get('success'):
                print(f"✅ {name}: Called {tool_call['function_name']}({tool_call['arguments']})")
            else:
                text = extract_text_response(response)
                print(f"❌ {name}: No tool called - {text[:60]}...")
        else:
            print(f"❌ {name}: Error - {response.get('error')}")

print("🧪 System prompt comparison ready!")

🧪 System prompt comparison ready!


---

## 3. Parameter Design That Works (7 mins)

Good parameter design helps the model extract the right information from user queries.

In [9]:
# Example: Different parameter designs for a unit converter

# Bad: Unclear parameters
bad_converter = {
    "name": "convert_units",
    "description": "Convert between different units of measurement",
    "parameters": {
        "type": "object",
        "properties": {
            "value": {
                "type": "number",
                "description": "Number to convert"
            },
            "from_unit": {
                "type": "string",
                "description": "Source unit"
            },
            "to_unit": {
                "type": "string",
                "description": "Target unit"
            }
        },
        "required": ["value", "from_unit", "to_unit"]
    }
}

# Good: Clear, specific parameters with examples
good_converter = {
    "name": "convert_units",
    "description": "Convert between different units of measurement for distance, weight, temperature, and volume",
    "parameters": {
        "type": "object",
        "properties": {
            "value": {
                "type": "number",
                "description": "The numerical value to convert (e.g., 5.5, 100, 32)"
            },
            "from_unit": {
                "type": "string",
                "description": "Source unit - use standard abbreviations like 'km', 'miles', 'kg', 'lbs', 'celsius', 'fahrenheit', 'liters', 'gallons'"
            },
            "to_unit": {
                "type": "string",
                "description": "Target unit - use same format as from_unit (e.g., 'km', 'miles', 'kg', 'lbs')"
            }
        },
        "required": ["value", "from_unit", "to_unit"]
    }
}

display_markdown("""
### Parameter Design Principles:

1. **Be Specific**: "Source unit" vs "use standard abbreviations like 'km', 'miles'"
2. **Provide Examples**: Show the format you expect
3. **Clarify Types**: Number vs string, required vs optional
4. **Consider Edge Cases**: What happens with unusual inputs?
""")


### Parameter Design Principles:

1. **Be Specific**: "Source unit" vs "use standard abbreviations like 'km', 'miles'"
2. **Provide Examples**: Show the format you expect
3. **Clarify Types**: Number vs string, required vs optional
4. **Consider Edge Cases**: What happens with unusual inputs?


In [10]:
# Test parameter design differences
queries = [
    "Convert 5 kilometers to miles",
    "How many pounds is 2.5 kilograms?",
]

system_prompt = "You are a helpful assistant with unit conversion tools. Use them when users ask for conversions."

for query in queries:
    print(f"\n📏 Testing: '{query}'")
    
    # Test with bad parameters
    response = call_openrouter(
        prompt=[{"role": "system", "content": system_prompt}, {"role": "user", "content": query}],
        model="openai/gpt-4o-2024-11-20",
        functions=[bad_converter],
        temperature=0.3,
        max_tokens=150
    )
    
    if response.get('success'):
        tool_call = extract_function_call(response)
        if tool_call.get('success'):
            print(f"❌ Bad params: {tool_call['arguments']}")
        else:
            print("❌ Bad params: No tool called")
    
    # Test with good parameters
    response = call_openrouter(
        prompt=[{"role": "system", "content": system_prompt}, {"role": "user", "content": query}],
        model="openai/gpt-4o-2024-11-20",
        functions=[good_converter],
        temperature=0.3,
        max_tokens=150
    )
    
    if response.get('success'):
        tool_call = extract_function_call(response)
        if tool_call.get('success'):
            print(f"✅ Good params: {tool_call['arguments']}")
        else:
            print("✅ Good params: No tool called")


📏 Testing: 'Convert 5 kilometers to miles'
❌ Bad params: {'value': 5, 'from_unit': 'kilometers', 'to_unit': 'miles'}
✅ Good params: {'value': 5, 'from_unit': 'km', 'to_unit': 'miles'}

📏 Testing: 'How many pounds is 2.5 kilograms?'
❌ Bad params: {'value': 2.5, 'from_unit': 'kilograms', 'to_unit': 'pounds'}
✅ Good params: {'value': 2.5, 'from_unit': 'kg', 'to_unit': 'lbs'}


---

## 4. System Prompts for Tool Guidance (8 mins)

System prompts set the context for how and when tools should be used.

In [11]:
# Different system prompt strategies

system_prompts = {
    "generic": "You are a helpful assistant.",
    
    "tool_aware": "You are a helpful assistant with access to weather and calculation tools. Use them when appropriate.",
    
    "specific_guidance": """
You are a helpful assistant with access to weather and calculation tools.

Use the weather tool when users:
- Ask about current conditions
- Want to plan outdoor activities
- Need weather-related advice

Use the calculator tool when users:
- Need mathematical calculations
- Ask for percentages, tips, or conversions
- Want to solve numerical problems

Always use tools when the information is needed to give an accurate answer.
""",
    
    "persona_based": """
You are an outdoor activity planning assistant. You help people make informed decisions about outdoor activities.

You have access to weather tools and calculators. Use weather tools to check conditions and calculators for any planning math (distances, costs, etc.).

Always check the weather when someone asks about outdoor activities, even if they don't explicitly mention weather.
"""
}

# Tools for testing
tools = [good_weather_tool, good_converter]

def compare_system_prompts(query, prompts_to_test):
    """Compare how different system prompts handle the same query"""
    print(f"\n🎯 Query: '{query}'")
    print("=" * 50)
    
    for name, prompt in prompts_to_test.items():
        response = call_openrouter(
            prompt=[{"role": "system", "content": prompt}, {"role": "user", "content": query}],
            model="google/gemini-2.5-flash-preview-05-20",
            functions=tools,
            temperature=0.3,
            max_tokens=200
        )
        
        if response.get('success'):
            tool_call = extract_function_call(response)
            if tool_call.get('success'):
                print(f"✅ {name}: Called {tool_call['function_name']}({tool_call['arguments']})")
            else:
                text = extract_text_response(response)
                print(f"❌ {name}: No tool called - {text[:60]}...")
        else:
            print(f"❌ {name}: Error - {response.get('error')}")

print("🧪 System prompt comparison ready!")

🧪 System prompt comparison ready!


In [12]:
# Demonstration of tool call extraction using Day 2 utilities

def demo_tool_extraction():
    """Demonstrate tool call extraction with a live example."""
    print("🧪 Testing tool call extraction...")
    
    # Simple test using just the API utilities from Day 2
    test_response = call_openrouter(
        prompt="What's the weather in Tokyo?",
        model="openai/gpt-4o-2024-11-20",
        functions=[{
            "name": "get_weather",
            "description": "Get current weather information for any city",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name (e.g., 'Tokyo', 'London')"
                    }
                },
                "required": ["location"]
            }
        }],
        temperature=0.7,
        max_tokens=200
    )
    
    if test_response.get("success"):
        # Use the Day 2 extraction function
        tool_call = extract_function_call(test_response)
        if tool_call.get('success'):
            print("\n🎯 Tool call extracted successfully!")
            
            # Display results with proper formatting
            tool_info = f"""
**Function Called:** `{tool_call['function_name']}`  
**Arguments:** `{tool_call['arguments']}`  
**Status:** Tool call detected and parsed correctly
            """
            display_markdown(tool_info, "Tool Call Extraction Results")
            
        else:
            regular_response = extract_text_response(test_response)
            display_llm_response(regular_response, title="Regular Response (No Tool Call)")
    else:
        display_error(f"API call failed: {test_response.get('error')}")

# Function is defined but not called automatically
print("🔧 Tool extraction demo function ready (call demo_tool_extraction() to test)")

🔧 Tool extraction demo function ready (call demo_tool_extraction() to test)


### Exercise: Write a System Prompt
Create a system prompt for a travel planning assistant that has access to weather and calculator tools.

In [13]:
# Exercise: Write a travel planning system prompt
travel_system_prompt = """
# TODO: Write your travel planning system prompt here
# Consider:
# - What's the assistant's role?
# - When should it use weather tools?
# - When should it use calculator tools?
# - What's the personality/tone?
"""

# Test your prompt
test_queries = [
    "I'm planning a trip to Barcelona next week. What should I pack?",
    "How much should I budget for a 5-day trip if hotels cost $120/night?"
]

# Uncomment to test:
# for query in test_queries:
#     test_tool_calling(query, travel_system_prompt, tools, f"Travel Assistant Test")

---

## 5. Multi-Tool Query Strategies (5 mins)

Some queries need multiple tools. Let's see how to handle them effectively.

In [19]:
# Note: This cell demonstrates multi-tool strategies but requires the enhanced function from cell 22
# Let's create a simple demonstration instead

def demo_multi_tool_strategies():
    """Demonstrate different strategies for multi-tool execution."""
    
    # Sample query that needs multiple tools
    sample_query = "I'm planning a trip to London. Tell me the weather and calculate 20% tip on £150."
    
    print(f"🎯 Sample Multi-Tool Query: '{sample_query}'")
    print("=" * 60)
    
    print("\n📋 **Strategy Analysis:**")
    print("1. **Generic**: Basic system prompt, may miss tools")
    print("2. **Specific**: Explicit tool usage instructions") 
    print("3. **Contextual**: Role-based prompts that encourage tool use")
    
    print("\n📝 **Key Strategies for Multi-Tool Queries:**")
    print("✅ Detect multiple needs in the query")
    print("✅ Use appropriate system prompts for each tool")
    print("✅ Execute tools in logical order") 
    print("✅ Combine results meaningfully")
    print("✅ Provide comprehensive final response")
    
    print("\n💡 **Note**: The enhanced implementation is in cell 22!")
    print("   Run that cell first to see the full multi-tool approach in action.")

# Run the demonstration
demo_multi_tool_strategies()

print("\n🔧 **Multi-Tool Strategy Framework Ready!**")
print("Move to cell 22 to see the full enhanced implementation.")

🎯 Sample Multi-Tool Query: 'I'm planning a trip to London. Tell me the weather and calculate 20% tip on £150.'

📋 **Strategy Analysis:**
1. **Generic**: Basic system prompt, may miss tools
2. **Specific**: Explicit tool usage instructions
3. **Contextual**: Role-based prompts that encourage tool use

📝 **Key Strategies for Multi-Tool Queries:**
✅ Detect multiple needs in the query
✅ Use appropriate system prompts for each tool
✅ Execute tools in logical order
✅ Combine results meaningfully
✅ Provide comprehensive final response

💡 **Note**: The enhanced implementation is in cell 22!
   Run that cell first to see the full multi-tool approach in action.

🔧 **Multi-Tool Strategy Framework Ready!**
Move to cell 22 to see the full enhanced implementation.


In [28]:
# Enhanced Multi-Tool Implementation Building on Notebook 2

# First, let's recreate the core tools from Notebook 2 with better location handling
def get_weather(location: str) -> str:
    """Get current weather for a location (simulated with better location matching)."""
    # Enhanced weather data with more locations
    weather_data = {
        "london": "15°C, Light rain, 80% humidity, Wind: 12 km/h SW",
        "tokyo": "22°C, Partly cloudy, 60% humidity, Wind: 8 km/h E", 
        "new york": "18°C, Overcast, 65% humidity, Wind: 15 km/h NW",
        "paris": "16°C, Cloudy, 70% humidity, Wind: 5 km/h N",
        "denver": "12°C, Clear sky, 45% humidity, Wind: 8 km/h E",
        "barcelona": "19°C, Sunny, 55% humidity, Wind: 10 km/h SE",
        "sydney": "25°C, Clear sky, 50% humidity, Wind: 12 km/h NE"
    }
    
    # Normalize location for better matching
    location_normalized = location.lower().strip()
    
    # Remove common suffixes and handle various formats
    location_clean = location_normalized
    for suffix in [", japan", ", uk", ", usa", ", france", ", spain", ", australia"]:
        location_clean = location_clean.replace(suffix, "")
    
    # Handle common alternate names
    location_aliases = {
        "nyc": "new york",
        "ny": "new york", 
        "la": "los angeles",
        "sf": "san francisco",
        "ldn": "london"
    }
    
    if location_clean in location_aliases:
        location_clean = location_aliases[location_clean]
    
    # Try exact match first
    if location_clean in weather_data:
        return f"Current weather in {location.title()}: {weather_data[location_clean]}"
    
    # Try partial matching for cities
    for city, weather in weather_data.items():
        if city in location_clean or location_clean in city:
            return f"Current weather in {location.title()}: {weather}"
    
    # If no match found, provide available cities
    available_cities = ", ".join([city.title() for city in weather_data.keys()])
    return f"Weather data not available for '{location}'. Available cities: {available_cities}"

def calculate(expression: str) -> str:
    """Safely evaluate mathematical expressions."""
    try:
        allowed_chars = set('0123456789+-*/()., ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic math operations allowed"
        result = eval(expression)
        if isinstance(result, float):
            if result.is_integer():
                result = int(result)
            else:
                result = round(result, 2)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

def extract_and_execute_tool_call(response, tool_functions):
    """Extract and execute tool calls from API response."""
    if not response.get("success"):
        return None, None
    
    try:
        message = response["response"]["choices"][0]["message"]
        tool_calls = message.get("tool_calls", [])
        
        if tool_calls:
            tool_call = tool_calls[0]
            function_name = tool_call["function"]["name"]
            arguments = json.loads(tool_call["function"]["arguments"])
            
            if function_name in tool_functions:
                # Handle different argument patterns for each function
                if function_name == "get_weather" and "location" in arguments:
                    result = tool_functions[function_name](arguments["location"])
                elif function_name == "calculate" and "expression" in arguments:
                    result = tool_functions[function_name](arguments["expression"])
                else:
                    # Fallback to keyword arguments
                    result = tool_functions[function_name](**arguments)
                return tool_call, result
    except Exception as e:
        return None, f"Error executing tool: {e}"
    
    return None, None

# Enhanced multi-tool approach with better prompts
def enhanced_multi_tool_approach(user_query, system_prompt_style="generic"):
    """Enhanced multi-tool approach with improved prompting strategies."""
    print(f"🤖 Enhanced approach ({system_prompt_style}): '{user_query}'")
    
    # Different prompt strategies
    prompt_strategies = {
        "generic": {
            "weather": "You are a helpful assistant. Use the weather tool when needed.",
            "calculator": "You are a helpful assistant. Use the calculator when needed.",
        },
        "specific": {
            "weather": "You are a weather assistant specialized in providing current conditions for outdoor planning. Always use the weather tool to get accurate, real-time weather data when users ask about weather or outdoor activities.",
            "calculator": "You are a calculation assistant focused on precision and accuracy. Use the calculator tool for any mathematical operations to ensure correct results, even for simple calculations.",
        },
        "contextual": {
            "weather": "You are an outdoor activity planning assistant. Weather conditions are crucial for planning, so always check weather when users mention locations or outdoor activities, even if they don't explicitly ask for weather.",
            "calculator": "You are a financial and planning assistant. Use calculations to help users with budgeting, planning, and decision-making. Always calculate precise numbers rather than estimates.",
        }
    }
    
    results = []
    strategy = prompt_strategies.get(system_prompt_style, prompt_strategies["generic"])
    
    # Enhanced weather detection - more comprehensive location detection
    weather_keywords = ["weather", "temperature", "rain", "sunny", "outdoor", "outside", "pack", "clothes", "jacket"]
    location_keywords = ["london", "tokyo", "paris", "new york", "denver", "barcelona", "sydney", "japan", "uk", "france", "spain"]
    
    if any(word in user_query.lower() for word in weather_keywords) or any(loc in user_query.lower() for loc in location_keywords):
        print(f"🌤️ Using {system_prompt_style} weather prompt...")
        
        weather_response = call_openrouter(
            prompt=[
                {"role": "system", "content": strategy["weather"]},
                {"role": "user", "content": user_query}
            ],
            model="google/gemini-2.5-flash-preview",
            functions=[{
                "name": "get_weather",
                "description": "Get current weather conditions including temperature, humidity, and precipitation for planning outdoor activities",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name (e.g., 'Tokyo', 'London', 'Paris'). You can include country for clarity but just the city name works too."
                        }
                    },
                    "required": ["location"]
                }
            }],
            temperature=0.3,
            max_tokens=200
        )
        
        if weather_response.get("success"):
            tool_call, result = extract_and_execute_tool_call(weather_response, {"get_weather": get_weather})
            if tool_call:
                results.append(("Weather", result))
                print(f"✅ Weather ({system_prompt_style}): {result}")
    
    # Calculator detection and execution  
    calc_keywords = ["calculate", "tip", "percent", "%", "cost", "budget", "math", "+", "-", "*", "/", "pace", "speed", "per day", "/day"]
    
    if any(word in user_query.lower() for word in calc_keywords) or any(char in user_query for char in "€$£¥"):
        print(f"🧮 Using {system_prompt_style} calculator prompt...")
        
        calc_response = call_openrouter(
            prompt=[
                {"role": "system", "content": strategy["calculator"]},
                {"role": "user", "content": user_query}
            ],
            model="google/gemini-2.5-flash-preview",
            functions=[{
                "name": "calculate",
                "description": "Perform mathematical calculations for planning, budgeting, and problem-solving with precise results",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "Mathematical expression to calculate (e.g., '25 * 3', '150 * 0.20', '5 / 25 * 60' for pace calculations)"
                        }
                    },
                    "required": ["expression"]
                }
            }],
            temperature=0.3,
            max_tokens=200
        )
        
        if calc_response.get("success"):
            tool_call, result = extract_and_execute_tool_call(calc_response, {"calculate": calculate})
            if tool_call:
                results.append(("Calculation", result))
                print(f"✅ Calculation ({system_prompt_style}): {result}")
    
    # Combine results with enhanced final prompt
    if results:
        print(f"📝 Combining {len(results)} results with {system_prompt_style} strategy...")
        
        results_text = "Information gathered:\n\n"
        for category, result in results:
            results_text += f"**{category}**: {result}\n\n"
        
        # Enhanced final prompts by strategy
        final_prompts = {
            "generic": f"User asked: '{user_query}'\n\n{results_text}\nPlease provide a helpful response.",
            "specific": f"User Query: '{user_query}'\n\n{results_text}\nBased on this precise information, provide a comprehensive and actionable response that directly addresses the user's needs.",
            "contextual": f"Planning Request: '{user_query}'\n\n{results_text}\nAs a planning assistant, synthesize this information to provide practical advice and recommendations that help the user make informed decisions."
        }
        
        final_response = call_openrouter(
            prompt=final_prompts[system_prompt_style],
            model="google/gemini-2.5-flash-preview",
            temperature=0.7,
            max_tokens=1000
        )
        
        if final_response.get("success"):
            return extract_text_response(final_response)
        else:
            return f"Error in final response: {final_response.get('error')}"
    else:
        return "No tools were triggered for this query."

print("🚀 Enhanced multi-tool approach ready with improved weather location matching!")
# result1 = enhanced_multi_tool_approach("What's the weather in Paris and what's 15% tip on €80?")
# print(result1)
result2 = enhanced_multi_tool_approach("Should I go running in Tokyo? Also calculate my pace if I run 5km in 25 minutes.", "contextual")
print(result2)

🚀 Enhanced multi-tool approach ready with improved weather location matching!
🤖 Enhanced approach (contextual): 'Should I go running in Tokyo? Also calculate my pace if I run 5km in 25 minutes.'
🌤️ Using contextual weather prompt...
✅ Weather (contextual): Current weather in Tokyo: 22°C, Partly cloudy, 60% humidity, Wind: 8 km/h E
🧮 Using contextual calculator prompt...
✅ Calculation (contextual): Result: 5
📝 Combining 2 results with contextual strategy...
Okay, let's break down your running plan for Tokyo!

Based on the information you provided:

**Should you go running in Tokyo?**

* **Weather:** 22°C with partly cloudy skies is generally a pleasant temperature for running. The humidity at 60% is moderate – not excessively high, but you'll likely feel it a bit. The wind at 8 km/h is light and shouldn't be a significant factor.
* **Overall:** The current weather conditions in Tokyo are quite suitable for a run. It's not too hot, and the humidity is manageable.

**Practical Advice and 

### Exercise: Multi-Tool Planning
Design a prompt strategy for this complex query: "Plan my outdoor lunch in Paris - I need to walk 2.5 km to get there and want to know if I should bring a jacket."

In [29]:
# Exercise: Handle the complex lunch planning query
lunch_query = "Plan my outdoor lunch in Paris - I need to walk 2.5 km to get there and want to know if I should bring a jacket."

# TODO: Design a system prompt that would handle this well
your_multi_tool_prompt = """
# Your prompt here - think about:
# - What tools are needed?
# - In what order?
# - How to combine the results?
"""

# Test it:
# test_tool_calling(lunch_query, your_multi_tool_prompt, [good_weather_tool, good_converter], "Your Multi-Tool Test")

---

## 6. Debugging and Iteration (2 mins)

Quick tips for diagnosing and fixing tool calling issues.

In [30]:
# Debugging checklist and common fixes

debugging_tips = """
# 🔧 Tool Calling Debugging Checklist

## When tools aren't being called:
1. **Check tool description** - Is it clear when to use this tool?
2. **Review system prompt** - Does it mention using tools?
3. **Test with explicit queries** - Try "Use the weather tool for London"
4. **Check parameter clarity** - Are parameter descriptions specific enough?

## When wrong parameters are extracted:
1. **Add examples** - Show the format you want in parameter descriptions
2. **Use enums** - For limited options, specify allowed values
3. **Improve descriptions** - Be more specific about expected format

## When tools fail in complex scenarios:
1. **Break down the query** - Test each tool individually
2. **Add guidance** - Tell the model the order to use tools
3. **Use examples** - Show multi-tool usage in system prompt

## Quick fixes that often work:
- Add "Use your tools when needed" to system prompt
- Include use cases in tool descriptions
- Provide format examples in parameter descriptions
- Test with simpler, more direct queries first
"""

display_markdown(debugging_tips)

# Example: Fix a problematic tool
def quick_fix_demo():
    # Before: Tool that often fails
    problematic_tool = {
        "name": "search_knowledge",
        "description": "Search for information",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"}
            },
            "required": ["query"]
        }
    }
    
    # After: Quick fixes applied
    fixed_tool = {
        "name": "search_knowledge",
        "description": "Search for factual information when you need to look up specific facts, dates, or details not in your training data",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string", 
                    "description": "Specific search query using keywords (e.g., 'Python pandas tutorial', 'Paris population 2024', 'quantum computing basics')"
                }
            },
            "required": ["query"]
        }
    }
    
    print("🔧 Applied quick fixes:")
    print("✅ Added use case to description")
    print("✅ Included format examples")
    print("✅ Specified when to use the tool")

quick_fix_demo()


# 🔧 Tool Calling Debugging Checklist

## When tools aren't being called:
1. **Check tool description** - Is it clear when to use this tool?
2. **Review system prompt** - Does it mention using tools?
3. **Test with explicit queries** - Try "Use the weather tool for London"
4. **Check parameter clarity** - Are parameter descriptions specific enough?

## When wrong parameters are extracted:
1. **Add examples** - Show the format you want in parameter descriptions
2. **Use enums** - For limited options, specify allowed values
3. **Improve descriptions** - Be more specific about expected format

## When tools fail in complex scenarios:
1. **Break down the query** - Test each tool individually
2. **Add guidance** - Tell the model the order to use tools
3. **Use examples** - Show multi-tool usage in system prompt

## Quick fixes that often work:
- Add "Use your tools when needed" to system prompt
- Include use cases in tool descriptions
- Provide format examples in parameter descriptions
- Test with simpler, more direct queries first


🔧 Applied quick fixes:
✅ Added use case to description
✅ Included format examples
✅ Specified when to use the tool


---

## 🎯 Summary

**Key Takeaways for Better Tool Calling:**

1. **Tool Descriptions Matter** - Be specific about what the tool does and when to use it
2. **Parameter Design** - Provide clear descriptions with examples and expected formats
3. **System Prompts** - Guide the model on when and how to use tools
4. **Multi-Tool Strategies** - Plan the sequence and combination of tool usage
5. **Debug Systematically** - Use the checklist to diagnose and fix issues

**Next Steps:**
- Practice with your own tools and queries
- Experiment with different prompt styles
- Build more complex multi-tool scenarios

---

### 💪 Final Challenge
Create a "Smart Travel Advisor" that uses weather, calculator, and converter tools to help plan a trip. Test it with: "I'm planning a 3-day trip to Tokyo. Should I pack warm clothes and how much will food cost at $25/day?"

In [31]:
# Final Challenge: Enhanced Smart Travel Advisor

# Enhanced travel tools building on Notebook 2's foundation
travel_tools = [
    {
        "name": "get_weather",
        "description": "Get current weather conditions including temperature, humidity, and precipitation for planning outdoor activities",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City name with country for accurate weather lookup (e.g., 'London, UK' or 'Tokyo, Japan')"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "calculate",
        "description": "Perform mathematical calculations for trip planning including costs, distances, and time estimates",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Mathematical expression to calculate (e.g., '3 * 25', '150 / 2', '(100 + 50) * 1.2')"
                }
            },
            "required": ["expression"]
        }
    }
]

def enhanced_travel_advisor(user_query, prompt_strategy="advanced"):
    """Enhanced travel advisor using improved prompting techniques."""
    
    # Advanced prompt strategies for travel planning
    strategies = {
        "basic": {
            "system": "You are a travel assistant. Use tools when needed.",
            "final": "Provide travel advice based on the information gathered."
        },
        "advanced": {
            "system": "You are an expert travel planning assistant specializing in practical, actionable advice. You have access to weather and calculation tools. Always check weather conditions for destinations and calculate precise costs, distances, and time estimates when planning trips. Focus on helping travelers make informed decisions.",
            "final": "As a travel expert, synthesize the gathered information to provide comprehensive, practical travel advice that addresses safety, comfort, and budget considerations."
        }
    }
    
    strategy = strategies.get(prompt_strategy, strategies["advanced"])
    print(f"🧳 Enhanced Travel Advisor ({prompt_strategy}): '{user_query}'")
    
    results = []
    
    # Enhanced weather detection for travel context
    weather_triggers = ["weather", "pack", "clothes", "temperature", "rain", "sunny", "climate"]
    location_keywords = ["tokyo", "london", "paris", "new york", "denver", "barcelona", "trip", "travel", "visit"]
    
    if any(word in user_query.lower() for word in weather_triggers + location_keywords):
        print("🌤️ Getting weather information for travel planning...")
        
        weather_response = call_openrouter(
            prompt=[
                {"role": "system", "content": f"{strategy['system']} When users mention travel destinations or ask about packing, always check weather to provide appropriate clothing and activity recommendations."},
                {"role": "user", "content": user_query}
            ],
            model="google/gemini-2.5-flash-preview-05-20",
            functions=[travel_tools[0]],
            temperature=0.3,
            max_tokens=200
        )
        
        if weather_response.get("success"):
            tool_call, result = extract_and_execute_tool_call(weather_response, {"get_weather": get_weather})
            if tool_call:
                results.append(("Weather", result))
                print(f"✅ Weather: {result}")
    
    # Enhanced calculation detection for travel costs
    calc_triggers = ["cost", "budget", "price", "money", "calculate", "total", "per day", "/day", "night", "meal"]
    
    if any(word in user_query.lower() for word in calc_triggers) or any(char in user_query for char in "€$£¥*+"):
        print("🧮 Calculating travel costs...")
        
        calc_response = call_openrouter(
            prompt=[
                {"role": "system", "content": f"{strategy['system']} When users ask about travel costs, budgets, or pricing, use the calculator to provide precise cost calculations and budget breakdowns."},
                {"role": "user", "content": user_query}
            ],
            model="google/gemini-2.5-flash-preview-05-20",
            functions=[travel_tools[1]],
            temperature=0.3,
            max_tokens=200
        )
        
        if calc_response.get("success"):
            tool_call, result = extract_and_execute_tool_call(calc_response, {"calculate": calculate})
            if tool_call:
                results.append(("Budget Calculation", result))
                print(f"✅ Calculation: {result}")
    
    # Enhanced final response generation
    if results:
        print(f"📝 Synthesizing {len(results)} pieces of travel information...")
        
        results_text = "Travel Information Gathered:\n\n"
        for category, result in results:
            results_text += f"**{category}**: {result}\n\n"
        
        enhanced_final_prompt = f"""
Travel Planning Request: "{user_query}"

{results_text}

{strategy['final']} Consider:
- Practical packing recommendations based on weather
- Budget-conscious advice based on calculations  
- Safety and comfort considerations
- Specific actionable next steps

Provide a complete, helpful response that addresses all aspects of the user's travel planning needs.
"""
        
        final_response = call_openrouter(
            prompt=enhanced_final_prompt,
            model="google/gemini-2.5-flash-preview-05-20",
            temperature=0.7,
            max_tokens=400
        )
        
        if final_response.get("success"):
            return extract_text_response(final_response)
        else:
            return f"Error in final response: {final_response.get('error')}"
    else:
        # Direct travel advice without tools
        direct_response = call_openrouter(
            prompt=[
                {"role": "system", "content": strategy["system"]},
                {"role": "user", "content": user_query}
            ],
            model="google/gemini-2.5-flash-preview-05-20",
            temperature=0.7,
            max_tokens=300
        )
        
        if direct_response.get("success"):
            return extract_text_response(direct_response)
        else:
            return f"Error: {direct_response.get('error')}"

# Test the enhanced travel advisor
print("🏆 **FINAL CHALLENGE: Enhanced Smart Travel Advisor**")
print("=" * 80)

challenge_query = "I'm planning a 3-day trip to London. Should I pack warm clothes and how much will food cost at $25/day?"

print("\n🧪 **Testing Enhanced vs Basic Approach:**")

print("\n📋 **BASIC APPROACH:**")
basic_result = enhanced_travel_advisor(challenge_query, "basic")
display_llm_response(basic_result, title="Basic Travel Advisor")

print("\n📋 **ENHANCED APPROACH:**") 
advanced_result = enhanced_travel_advisor(challenge_query, "advanced")
display_llm_response(advanced_result, title="Enhanced Travel Advisor")

print("\n🎯 **Key Improvements Demonstrated:**")
display_markdown("""
1. **Better Tool Selection**: Enhanced prompts lead to more relevant tool usage
2. **Improved Parameter Quality**: More specific, accurate tool parameters  
3. **Contextual Understanding**: Better recognition of when tools are needed
4. **Enhanced Final Synthesis**: More comprehensive and actionable advice
5. **Multi-Tool Coordination**: Smooth integration of multiple tool results

**This shows how Notebook 3's prompt engineering techniques enhance Notebook 2's multi-tool foundation!**
""", "Learning Outcomes")

print("🚀 Enhanced travel advisor ready! This demonstrates the full power of combining")
print("   Notebook 2's multi-tool execution with Notebook 3's prompt engineering!")

🏆 **FINAL CHALLENGE: Enhanced Smart Travel Advisor**

🧪 **Testing Enhanced vs Basic Approach:**

📋 **BASIC APPROACH:**
🧳 Enhanced Travel Advisor (basic): 'I'm planning a 3-day trip to London. Should I pack warm clothes and how much will food cost at $25/day?'
🌤️ Getting weather information for travel planning...
✅ Weather: Current weather in London, Uk: 15°C, Light rain, 80% humidity, Wind: 12 km/h SW
🧮 Calculating travel costs...
✅ Calculation: Result: 75
📝 Synthesizing 2 pieces of travel information...


Here's your personalized travel advice for your 3-day trip to London, based on the information you've provided!

## Your London Trip: Practical Advice for a Great Experience

### Packing for London Weather: Yes, Pack Warm (and Smart!)

Given the current weather in London (15°C, light rain, humid, and windy), here's what you should definitely pack to stay comfortable:

*   **Warm Layers are Key:** While 15°C isn't freezing, it's definitely cool, especially with the wind and humidity. Think in layers:
    *   **Base Layer:** Long-sleeved t-shirts or light sweaters.
    *   **Mid-Layer:** A fleece jacket, a thicker sweater, or a cardigan.
    *   **Outer Layer:** A **waterproof and windproof jacket** is absolutely essential. This will be your best friend against the rain and wind.
*   **Bottoms:** Jeans, comfortable trousers, or even thicker leggings will be suitable.
*   **Footwear:** **Comfortable, waterproof walking shoes or boots** are non-negotiable. You'll be doing a lot of walking, and wet feet are miserable. Avoid open-toed shoes or sandals.
*   **Accessories:**
    *   **Umbrella:** A compact, sturdy umbrella is a must-have.
    *   **Scarf:** A warm scarf will protect your neck from the wind and add an extra layer of warmth.
    *   **Light gloves:** Depending on your personal tolerance for cold, light gloves might be appreciated, especially in the evenings.
*   **Don't Forget:** Regular travel essentials like toiletries, chargers, adapters (UK plug type G), and any medications.

**In short: Yes, pack warm clothes, but prioritize layers and waterproof outer gear!**

### Food Budget: Your $25/


📋 **ENHANCED APPROACH:**
🧳 Enhanced Travel Advisor (advanced): 'I'm planning a 3-day trip to London. Should I pack warm clothes and how much will food cost at $25/day?'
🌤️ Getting weather information for travel planning...
✅ Weather: Current weather in London, Uk: 15°C, Light rain, 80% humidity, Wind: 12 km/h SW
🧮 Calculating travel costs...
✅ Calculation: Result: 75
📝 Synthesizing 2 pieces of travel information...


Here's a comprehensive guide for your 3-day trip to London, addressing your questions about packing and food costs, along with essential safety and comfort tips!

### London Trip: Packing, Budget, Safety & Comfort

**1. Packing for London's Weather (Practical & Comfortable)**

Given the current London weather (15°C, light rain, 80% humidity, 12 km/h SW wind), **yes, you should definitely pack warm clothes, but also be prepared for a mix of conditions.** London weather is notoriously unpredictable, even within a single day!

*   **Layers are Key:** This is the most crucial advice for London. You'll want to be able to add or remove clothing as the temperature fluctuates and you move between indoor attractions and outdoor sightseeing.
    *   **Base Layers:** Long-sleeved t-shirts or light sweaters.
    *   **Mid-Layers:** A fleece jacket, cardigan, or a thicker sweater.
    *   **Outer Layer:** A **waterproof and windproof jacket** is non-negotiable. Look for something that's breathable but will keep you dry. A packable down jacket could also be a good option if it's waterproof.
*   **Bottoms:** Jeans, comfortable trousers, or even warm leggings are suitable. Avoid anything too thin or easily soaked.
*   **Footwear:**
    *   **Comfortable, waterproof walking shoes or boots** are essential. You'll be doing a lot of walking! Make sure they have good grip.
    *   Bring an extra pair of socks in case yours get wet.
*   **Accessories:**
    *   **Umbrella:** A compact, sturdy umbrella is a must-have.
    *   **Scarf:** A warm scarf can make a big difference in keeping you comfortable, especially with the wind.
    *   **Beanie/


🎯 **Key Improvements Demonstrated:**


### Learning Outcomes


1. **Better Tool Selection**: Enhanced prompts lead to more relevant tool usage
2. **Improved Parameter Quality**: More specific, accurate tool parameters  
3. **Contextual Understanding**: Better recognition of when tools are needed
4. **Enhanced Final Synthesis**: More comprehensive and actionable advice
5. **Multi-Tool Coordination**: Smooth integration of multiple tool results

**This shows how Notebook 3's prompt engineering techniques enhance Notebook 2's multi-tool foundation!**


🚀 Enhanced travel advisor ready! This demonstrates the full power of combining
   Notebook 2's multi-tool execution with Notebook 3's prompt engineering!
